# Tutorial of PLAN for continous waveform

This notebook demonstrates the use of PLAN for performing the earthquake signal detection and phase (P & S) picking, association and location on continuous data. Here, we demonstrates the results using Ridgecrest squence.

In [1]:
%load_ext autoreload
%autoreload 2

# 1. import Lib 

In [2]:
import sys   
sys.path.append("..")
from utils.utils import *
from utils.detect_peaks import *
from utils.model_ridgecrest_vision import *
from utils.continue_dataloader import *


In [3]:
setup_seed(20)

# 2. import Data and construct DataLoader

In [4]:
# We utilize 16 stations in CI Network for evaluation the performance of PLAN in Ridgecrest Squence Detection.
station_file_path = "../data/gmap-stations.txt"
station_pandas = pd.read_csv(station_file_path, sep='|')
station_pandas = station_pandas.drop([0])
station_pandas

,#Network,Station,Latitude,Longitude,Elevation,Sitename,StartTime,EndTime
1,CI,CCC,35.52495,-117.36453,670.0,Christmas Canyon China Lake,2001-06-22T00:00:00,3000-01-01T00:00:00
2,CI,CLC,35.81574,-117.59751,775.0,China Lake,1948-07-08T00:00:00,3000-01-01T00:00:00
3,CI,DAW,36.27148,-117.59214,1477.4,Darwin,2017-02-27T00:00:00,3000-01-01T00:00:00
4,CI,JRC2,35.98249,-117.80885,1469.0,Joshua Ridge: China Lake,2004-01-15T00:00:00,3000-01-01T00:00:00
5,CI,LRL,35.47954,-117.68212,1340.0,Laurel Mtn,1992-07-29T00:00:00,3000-01-01T00:00:00
6,CI,MPM,36.05799,-117.48901,1839.0,Manuel Prospect Mine,1998-06-25T00:00:00,3000-01-01T00:00:00
7,CI,SLA,35.89095,-117.28332,1174.0,Slate Mountain,1999-03-09T00:00:00,3000-01-01T00:00:00
8,CI,SRT,35.69235,-117.75051,667.0,Snort,1982-07-31T00:00:00,3000-01-01T00:00:00
9,CI,TOW2,35.80856,-117.76488,685.0,Tower 2,2014-07-10T00:00:00,3000-01-01T00:00:00
10,CI,WBM,35.60839,-117.89049,892.0,Bowman Road,1979-09-26T00:00:00,3000-01-01T00:00:00


In [5]:
data_file = '../data/hn5day/20190704173000.CI.'
inputdata = load_continous_data(station_file_path,data_file,data_length = 3600)
print(inputdata.shape)

16it [00:20,  1.28s/it]

(16, 3, 360001)


Define Dataloader

In [6]:
# start_time means how long after 17:30:00. Since the data is 100Hz sample, 30000 means 5 minute (5*60*100)
# end_time means how long after 17:30:00.
# interval means shift window when deal with continous data. (Here is 5 seconds)
# Here test start_time is 17:35:00, end_time is 17:41:20
ridge_loader,batch_start_time = construct_dataloader(inputdata,station_file_path,
                     start_time = 30000,
                     end_time = 70000,
                     interval = 500,
                     batchsize = 1,
                     num_workers = 1)

In [27]:
# for mydata in ridge_loader:
#     print(mydata)

# 3. Load Model

In [7]:
# if you have gpu, you can change this sentence as device = torch.device('cuda')
device = torch.device('cpu')
model_PLAN = Main_GCNN('Trans').to(device)
load_model_name = '../model/model_PLAN_Ridge_continue.pt'
model_PLAN = load_model(load_model_name,model_PLAN)


# 4. Pred 

This function is corresponding with the continous workflow:
![示例图片](../assets/Continous_workflow.jpg)

In different region, you must change stack_value and value (Unfornately, it is a little trick and need more experience). For example, you can define P_stack_value by 0.15 times number of stations and define S_stack_value by 0.075 times number of stations.

In [8]:
earthquake_time_list_p,earthquake_time_list_s,earthquake_time_list_total,earthquake_loc_list = pred(model_PLAN,
                                                                                                    ridge_loader,
                                                                                                    station_file_path,
                                                                                                    device,
                                                                                                    batch_start_time,
                                                                                                    P_stack_value = 2.4,
                                                                                                    S_stack_value = 1.2, 
                                                                                                    P_value = 0.24, 
                                                                                                    S_value = 0.12, 
                                                                                                    time_sample = 200, 
                                                                                                    station_num = 4)

100%|█████████████████████████████████████████| 80/80 [00:35<00:00,  2.27it/s]


After test, the break time calculate by s wave is more accuracy. Therefore, in this case, we utilized earthquake_time_list_s. For general, we can use the average time of P/S wave.

# 5. Save Catalog 

In [9]:
# delete duplicate event within 2s
merged_time_series, merged_coordinates = merge_time_series(earthquake_time_list_s, earthquake_loc_list,time_threshold = 2)
final_catalog = np.concatenate([np.array(merged_time_series).reshape(-1,1),merged_coordinates],axis=-1)
formatted_data = np.array([final_catalog[:,0]] + [final_catalog[:,i].astype(float).round(4) for i in range(1, final_catalog.shape[1])]).T
formatted_data = formatted_data[:,0:3]
# save catalog
np.savetxt('test_catalog.txt', formatted_data, fmt='%s', delimiter=', ')